In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from datetime import datetime as dt
import os

def getNow(folder='logFolder'):
    now       = dt.now().strftime('%Y-%m-%d--%H-%M-%S')
    nowFolder = os.path.join('./{}'.format(folder), now)
    return nowFolder

In [2]:
N = 1000
X = np.random.rand(N, 2)
y = 2*X[:, 0] + 3*X[:, 1] + 4 + 0.1*(np.random.rand(N)-0.5)
y = y.reshape((-1, 1))

### Loading a previous graph

In [3]:
tf.reset_default_graph()
savePath = './models/2019-05-10--16-40-09/model.ckpt'
saver = tf.train.import_meta_graph(savePath + '.meta')

In [4]:
summaryFolder = getNow()

with tf.Session() as sess:
    writer = tf.summary.FileWriter(summaryFolder, sess.graph)
    writer.close()
    
print('tensorboard --logdir={}'.format( summaryFolder ))

tensorboard --logdir=./logFolder/2019-05-10--16-47-50


### Without loading preivous state

In [2]:
tf.reset_default_graph()
savePath = './models/2018-06-28--15-49-55/model.ckpt'
saver = tf.train.import_meta_graph(savePath + '.meta')

In [5]:
with tf.Session() as sess:
    sess.run('misc/init')
    errVal = sess.run( 'model/err:0', feed_dict={
            'io/inp:0' : X, 'io/out:0' : y})

print(errVal)

2.7633426


### Run the optimizer

In [6]:
with tf.Session() as sess:
    sess.run('misc/init')
    for i in range(1000):
        sess.run( 'optimizers/Adam', feed_dict={
            'io/inp:0' : X, 'io/out:0' : y})
        
    errVal = sess.run( 'model/err:0', feed_dict={
            'io/inp:0' : X, 'io/out:0' : y})

print(errVal)

0.0008152597


### After loading previous state

In [9]:
tf.reset_default_graph()
savePath = './models/2019-05-10--16-40-09/model.ckpt'
saver = tf.train.import_meta_graph(savePath + '.meta')

In [10]:
with tf.Session() as sess:
    sess.run('misc/init')
    saver.restore(sess, savePath)
    errVal = sess.run( 'model/err:0', feed_dict={
            'io/inp:0' : X, 'io/out:0' : y})
    
print(errVal)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./models/2019-05-10--16-40-09/model.ckpt
0.0008173431


### Get variables

In [12]:
tf.reset_default_graph()
savePath = './models/2019-05-10--16-40-09/model.ckpt'
saver = tf.train.import_meta_graph(savePath + '.meta')

In [13]:
graph = tf.get_default_graph()
err   = graph.get_tensor_by_name('model/err:0')
inp   = graph.get_tensor_by_name('io/inp:0')
out   = graph.get_tensor_by_name('io/out:0')

with tf.Session() as sess:
    sess.run('misc/init')
    saver.restore(sess, savePath)
    errVal = sess.run( err, feed_dict={
            inp : X, out : y})
    
print(errVal)

INFO:tensorflow:Restoring parameters from ./models/2019-05-10--16-40-09/model.ckpt
0.0008173431


### Using the graph collections

In [15]:
graph = tf.get_default_graph()
variables = graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

In [16]:
[v.name  for v in variables]

['model/layer_00000/W:0',
 'model/layer_00000/b:0',
 'model/layer_00001/W:0',
 'model/layer_00001/b:0',
 'model/layer_00002/W:0',
 'model/layer_00002/b:0']

In [17]:
[v.name for v in graph.get_collection(tf.GraphKeys.VARIABLES)]

Instructions for updating:
Use `tf.GraphKeys.GLOBAL_VARIABLES` instead.


['model/layer_00000/W:0',
 'model/layer_00000/b:0',
 'model/layer_00001/W:0',
 'model/layer_00001/b:0',
 'model/layer_00002/W:0',
 'model/layer_00002/b:0',
 'optimizers/beta1_power:0',
 'optimizers/beta2_power:0',
 'optimizers/model/layer_00000/W/Adam:0',
 'optimizers/model/layer_00000/W/Adam_1:0',
 'optimizers/model/layer_00000/b/Adam:0',
 'optimizers/model/layer_00000/b/Adam_1:0',
 'optimizers/model/layer_00001/W/Adam:0',
 'optimizers/model/layer_00001/W/Adam_1:0',
 'optimizers/model/layer_00001/b/Adam:0',
 'optimizers/model/layer_00001/b/Adam_1:0',
 'optimizers/model/layer_00002/W/Adam:0',
 'optimizers/model/layer_00002/W/Adam_1:0',
 'optimizers/model/layer_00002/b/Adam:0',
 'optimizers/model/layer_00002/b/Adam_1:0']

### Getting the variable values

In [18]:
with tf.Session() as sess:
    saver.restore(sess, savePath)
    for v in graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
        print(v.name)
        print(sess.run(v))

INFO:tensorflow:Restoring parameters from ./models/2019-05-10--16-40-09/model.ckpt
model/layer_00000/W:0
[[ 0.7627492  -0.00313599  0.3321067   0.46822044  0.06968354  0.36907607
   0.3505179   0.13753852  0.09524152  0.28487554]
 [ 0.75459844 -0.02262615  0.60972726  0.62974614  0.4918534   0.3288203
   0.27999717  0.21252342  0.68116593  0.47014427]]
model/layer_00000/b:0
[[0.30853215 0.37037417 0.35271388 0.34599015 0.3488921  0.3471873
  0.3521927  0.3590715  0.3331551  0.34924993]]
model/layer_00001/W:0
[[-4.23153155e-02 -1.51918232e-01  5.67491353e-02  4.97146398e-01
   3.27696145e-01]
 [ 4.43276197e-01  7.92495131e-01  4.50465530e-01  6.66887820e-01
   8.89209449e-01]
 [ 2.97447681e-01  7.85757542e-01  5.33800840e-01  5.46275496e-01
   3.97464544e-01]
 [ 4.58038628e-01  6.31498992e-01  1.50634989e-01  1.72377691e-01
   6.55393124e-01]
 [ 9.04999793e-01  1.00471884e-01  3.95871907e-01  9.87170413e-02
  -7.67018530e-04]
 [ 4.95717168e-01  4.09693539e-01  2.68080175e-01  4.18527961

## Transfer Learning

In [20]:
tf.reset_default_graph()
savePath = './models/2019-05-10--16-40-09/model.ckpt'
saver = tf.train.import_meta_graph(savePath + '.meta')

In [21]:
graph = tf.get_default_graph()
err   = graph.get_tensor_by_name('model/err:0')
inp   = graph.get_tensor_by_name('io/inp:0')
out   = graph.get_tensor_by_name('io/out:0')

In [22]:
for v in graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(v.name)

model/layer_00000/W:0
model/layer_00000/b:0
model/layer_00001/W:0
model/layer_00001/b:0
model/layer_00002/W:0
model/layer_00002/b:0


In [23]:
varsToTrain = [v for v in graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES) if 'model/layer_00002' in v.name]
with tf.variable_scope('new_opt'):
    opt = tf.train.AdamOptimizer(learning_rate=0.01).minimize(err, var_list=varsToTrain)
    
with tf.variable_scope('new_misc'):
    init  = tf.global_variables_initializer()

Instructions for updating:
Colocations handled automatically by placer.


In [24]:
summaryFolder = getNow()

with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, savePath)
    writer = tf.summary.FileWriter(summaryFolder, sess.graph)
    
    for i in range(1000):
        sess.run(opt, feed_dict={inp: X, out: y})
        
    for v in graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
        print('-----[{:30s}]--------'.format(v.name))
        print(v.eval())
    
    writer.close()
    
print('tensorboard --logdir={}'.format( summaryFolder ))

INFO:tensorflow:Restoring parameters from ./models/2019-05-10--16-40-09/model.ckpt
-----[model/layer_00000/W:0         ]--------
[[ 0.7627492  -0.00313599  0.3321067   0.46822044  0.06968354  0.36907607
   0.3505179   0.13753852  0.09524152  0.28487554]
 [ 0.75459844 -0.02262615  0.60972726  0.62974614  0.4918534   0.3288203
   0.27999717  0.21252342  0.68116593  0.47014427]]
-----[model/layer_00000/b:0         ]--------
[[0.30853215 0.37037417 0.35271388 0.34599015 0.3488921  0.3471873
  0.3521927  0.3590715  0.3331551  0.34924993]]
-----[model/layer_00001/W:0         ]--------
[[-4.23153155e-02 -1.51918232e-01  5.67491353e-02  4.97146398e-01
   3.27696145e-01]
 [ 4.43276197e-01  7.92495131e-01  4.50465530e-01  6.66887820e-01
   8.89209449e-01]
 [ 2.97447681e-01  7.85757542e-01  5.33800840e-01  5.46275496e-01
   3.97464544e-01]
 [ 4.58038628e-01  6.31498992e-01  1.50634989e-01  1.72377691e-01
   6.55393124e-01]
 [ 9.04999793e-01  1.00471884e-01  3.95871907e-01  9.87170413e-02
  -7.670